In [2]:
import numpy as np
import pandas as pd

# Trade-Offs in Missing Data Conventions

There are a number of schemes that have been developed to indicate the presence of missing data in a table or DataFrame. Generally, they revolve around one of two strategies:
- using a mask that globally indicates missing values, or 
- choosing a sentinel value that indicates a missing entry.


In the masking approach, the mask might be an entirely separate Boolean array, or it may involve appropriation of one bit in the data representation to locally indicate the null status of a value.

In the sentinel approach, the sentinel value could be some data-specific convention, such as indicating a missing integer value with -9999 or some rare bit pattern, or it could be a more global convention, such as indicating a missing floating-point value with NaN (Not a Number), a special value which is part of the IEEE floating-point specification.

None of these approaches is without trade-offs: 

- use of a **separate mask array** requires allocation of an additional Boolean array, which adds overhead in both storage and computation.
- A **sentinel value** reduces the range of valid values that can be represented, and may require extra (often non-optimized) logic in CPU and GPU arithmetic. Common special values like NaN are not available for all data types.

# Missing Data in Pandas

The way in which Pandas handles missing values is **constrained by its reliance on the NumPy package**, which does **not have a built-in notion of NA values for non-floating-point** data types.

While R contains four basic data types, NumPy supports far more than this: for example, while R has a single integer type, NumPy supports fourteen basic integer types once you account for available precisions, signedness, and endianness of the encoding. Reserving a specific bit pattern in all available NumPy types would lead to an unwieldy amount of overhead in special-casing various operations for various types, likely even requiring a new fork of the NumPy package. Further, for the smaller data types (such as 8-bit integers), sacrificing a bit to use as a mask will significantly reduce the range of values it can represent.

**NumPy does have support for masked arrays** – that is, arrays that have a separate Boolean mask array attached for marking data as "good" or "bad." Pandas could have derived from this, but the overhead in both storage, computation, and code maintenance makes that an unattractive choice.

# The choice of pandas for missing data 

With these constraints in mind, **Pandas chose to use sentinels for missing data**, and further chose to use two already-existing Python null values: 
- the special floating-point **NaN value,** 
- **and the Python None object**.
This choice has some side effects, as we will see, but in practice ends up being a good compromise in most cases of interest.

# None: Pythonic missing data

The first sentinel value used by Pandas is **None, a Python singleton object** that is often used for missing data in Python code. Because it is a Python object, None cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type 'object' (i.e., arrays of Python objects):

In [4]:
vals1 = np.array([1, 2, None, 3])
vals1

array([1, 2, None, 3], dtype=object)

In [6]:
vals2 = np.array([1, 2, 4, 3])
vals2.dtype

dtype('int64')

The **dtype=object** for vals1 means that the *best common type representation NumPy could infer for the contents of the array is that they are Python objects*.

While this kind of object array is useful for some purposes, any operations on the data will be done at the Python level, with much more overhead than the typically fast operations seen for arrays with native types:

In [8]:
for dtype in ['object','int']:
    print('dtype=',dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype= object
82.9 ms ± 3.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype= int
1.58 ms ± 46.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)



In [10]:
# vals1.sum()

# error: 
# TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'